## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
from IPython.display import clear_output
import time
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.drop("Unnamed: 0", axis=1, inplace=True)
city_data_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Weather_Description
0,new norfolk,-42.78,147.06,51.01,58,20,3.00,AU,2020-10-24 22:52:05,few clouds
1,bluff,-46.60,168.33,65.32,57,91,18.86,NZ,2020-10-24 22:52:05,overcast clouds
2,pevek,69.70,170.31,27.32,91,0,13.42,RU,2020-10-24 22:49:12,clear sky
3,gambela,8.25,34.58,70.63,88,26,4.09,ET,2020-10-24 22:52:06,scattered clouds
4,margate,51.38,1.39,55.00,86,94,7.00,GB,2020-10-24 22:49:39,moderate rain


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
def temperature_input():
    try:
        min_temp, max_tep = input("Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':\n ").strip().split(",")
    except ValueError:
        clear_output()
        print("\nPlease Enter 2 integers seperated by a ','")
        time.sleep(3)
        clear_output()
        min_temp, max_temp = temperature_input()
    return min_temp, max_tep


min_temp, max_temp = temperature_input()
   

Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':
 60,80


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Max_Min_Temp_df=city_data_df.loc[(city_data_df.Max_Temp >= int(min_temp)) & (city_data_df.Max_Temp <= int(max_temp))]

In [31]:
# 4a. Determine if there are any empty rows.
Max_Min_Temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 823 entries, 1 to 1849
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 823 non-null    object 
 1   Lat                  823 non-null    float64
 2   Lng                  823 non-null    float64
 3   Max_Temp             823 non-null    float64
 4   Humidity             823 non-null    int64  
 5   Cloudiness           823 non-null    int64  
 6   Wind_Speed           823 non-null    float64
 7   Country              821 non-null    object 
 8   Date                 823 non-null    object 
 9   Weather_Description  823 non-null    object 
dtypes: float64(4), int64(2), object(4)
memory usage: 70.7+ KB


In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Max_Min_Temp_df=Max_Min_Temp_df.dropna()

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Max_Min_Temp_df[["City", "Country", "Max_Temp", "Weather_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max_Temp,Weather_Description,Lat,Lng,Hotel Name
1,bluff,NZ,65.32,overcast clouds,-46.60,168.33,
3,gambela,ET,70.63,scattered clouds,8.25,34.58,
8,bondoukou,CI,73.65,scattered clouds,8.03,-2.80,
10,hermanus,ZA,62.01,scattered clouds,-34.42,19.23,
12,avarua,CK,77.00,broken clouds,-21.21,-159.78,
15,nishihara,JP,71.60,few clouds,26.18,127.76,
16,ukiah,US,73.99,clear sky,39.15,-123.21,
20,cabedelo,BR,77.00,scattered clouds,-6.98,-34.83,
21,alice springs,AU,64.40,clear sky,-23.70,133.88,
24,sidi ali,DZ,60.57,clear sky,36.10,0.42,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
counter=0
index_2=0
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Add lat and lng to params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f'Success {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df)*100))}%')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        
    except IndexError:
        print(f'\nFaliure {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df.index)*100))}% \n\t\tSkipping...\n')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
print("-"*30+"\nCompleted\t100.00%\n"+"-"*30)

Success 820 of 821 | 99.88%
------------------------------
Completed
------------------------------


In [59]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna().drop_duplicates()
type(clean_hotel_df)

pandas.core.frame.DataFrame

In [60]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:

### Heat_Layer_Map ###


# 11a. Add a marker layer for each city to the map. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description and Max Temp</dt><dd>{Weather_Description} \tMax Temp: {Max_Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max_Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:

### Marker_Layer_Map ###


# 11a. Add a marker layer for each city to the map. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description and Max Temp</dt><dd>{Weather_Description} \tMax Temp: {Max_Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max_Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5,map_type="HYBRID")
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))